Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Цель исследования

Наша цель - проанализировать  информацию о книгах, издательствах и авторах, а затем - выдать рекомендации коллегам

## Загрузка и знакомство с данными

Для начала импортируем нужные библиотеки и создадим подключение к БД

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-final-project-db'}

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db']) 

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Теперь познакомиимся с данными, но сначала пропишем функцию, которая на вход будет принимать запрос, а выводить его результат

In [5]:
def get(sql):
    return pd.io.sql.read_sql(sql, con = engine)

In [6]:
tbooks = (
"""
    SELECT * FROM books
""")
get(tbooks).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
tauthors = (
"""
    SELECT * FROM authors
""")
get(tauthors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
tpublishers = (
"""
    SELECT * FROM publishers
""")
get(tpublishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
tratings = (
"""
    SELECT * FROM ratings
""")
get(tratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
treviews = (
"""
    SELECT * FROM reviews 
""")
get(treviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Для исследования было получено 5 таблиц, которые содержат информацию о книгах, авторах, издательствах, а таже о пользователях, которые оставили свои оценки и отзывы по книгам. Дополнительно имеем схему данных, на которой предсставлены взаимоотношения между таблицами (ссылка на схему https://drive.google.com/file/d/1drMk6imrM1DTnpOZhb1txoOszHUQ3Fii/view?usp=sharing)

## Количество вышедших после 1 января 2000 года книг

In [11]:
query1 = (
"""
    SELECT COUNT(book_id) AS book_cnt
    FROM books
    WHERE publication_date >= '2000-01-01'
"""
)
get(query1)

,book_cnt
0,821


Если бы у нас были данные по выручке, которую сгенерировали продажи книг, мы могли бы провести дополнительный анализ. А так можем только сказать, что в среднем за 21 год сервис выпускал по 39 уникальных книг. Имеем ввиду уникальное сочетание автора и названия, без учета количества экземпляров

## Количество обзоров и средняя оценка для каждой книги 

In [12]:
query2 = (
"""
    SELECT
        books.book_id AS book_id,
        books.title,
        COUNT(DISTINCT(review_id)) AS review_cnt,
        AVG(rating) AS average_rating
    FROM 
        books
        LEFT JOIN ratings ON ratings.book_id = books.book_id
        LEFT JOIN reviews ON reviews.book_id = books.book_id
    GROUP BY
        books.book_id,
        books.title
    ORDER BY
        average_rating DESC
"""
)
get(query2)

,book_id,title,review_cnt,average_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


Как вариант, можем предложить коллегам издать дополнительную информацию про те книги, которые набрали максимальные средние оценки. А затем предложить ее нашим читателям

## Издательства, которые выпустили наибольшее число книг толще 50 страниц 

In [13]:
query3 = (
"""
    SELECT
        publisher,
        COUNT(title) AS cnt
    FROM 
        books
        LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
    WHERE num_pages > 50
    GROUP BY
        publisher
    ORDER BY 
        cnt DESC
    LIMIT
        5
        
"""
)
get(query3)

,publisher,cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Однозначно стоит начать переговоры с самыми популярными издательствами, чтобы иметь право распространять те новинки, которые у них появляются

## Авторы с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)

In [14]:
query4 = (
"""
    SELECT
        SUBQ.author AS author,
        AVG(SUBQ.avg_rating) AS avg_rating
    FROM (
        SELECT
            DISTINCT books.title AS title,
            AVG(ratings.rating) AS avg_rating,
            COUNT(ratings.rating) AS cnt_rating,
            books.author_id AS author_id,
            authors.author AS author
        FROM
            books
            LEFT JOIN ratings ON ratings.book_id = books.book_id
            LEFT JOIN authors ON authors.author_id = books.author_id
        GROUP BY
            books.title,
            books.author_id,
            authors.author
        HAVING
            COUNT(ratings.rating) > 50
        ) AS SUBQ
    GROUP BY
        author
    ORDER BY
        avg_rating DESC
    LIMIT
        5
"""
)
get(query4)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Arthur Golden,4.107143


А если нам удастся сочетать лучшие издательства и самых популярных авторов - наши дела 100% пойдут в гору

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [15]:
query5 = (
"""
    SELECT
        AVG(cnt_reviews) AS avg_cnt_reviews
    FROM
        (
            (SELECT
                username,
                COUNT(DISTINCT rating_id) AS cnt_rating
            FROM
                ratings
            GROUP BY
                username) AS SUBQ1
                LEFT JOIN (
                SELECT
                    username,
                    COUNT(review_id) AS cnt_reviews
                FROM
                    reviews
                GROUP BY
                    username) AS SUBQ2 ON SUBQ1.username = SUBQ2.username)
    WHERE
        cnt_rating > 50
"""
)
get(query5)

,avg_cnt_reviews
0,24.333333


И конечно же, мы выберем самых активных пользователей, которые пишут отзывы и ставят оценки: влияя на увеличение их активности, мы тем самым сможем обратить внимание новых читателей на новые книги и наш сервис.

## Рекомендации

В идеале нам стоит стартовать с продаж книг, которые удовлетворяют 3 условиям:
- средняя оценка в топе 3 раздела (средняя оценка для каждой книги)
- выпущены топовыми издательствами (4 раздел)
- написаны самыми популярными авторами (5 раздел)

Дополнтельно хочется отметить, что наш анализ был бы более полным, если у нас была финансовая информация: сколько стоила каждая книга, в каком количестве экземпляров она была выпущена и тд